## 1, Import Library

In [1]:
from machine_lib_output import *
from capm_alpha import *
from capm_alpha_opt import *

## 2, 登录
<div style="margin-left: 20px;">
1, 在machine_lib文件的login方法中填写用户名和密码后保存，然后来到本文件Restart Kernal后重新import machine_lib后才在本文件生效
</div>

<div style="margin-left: 20px;">
2, 打印INVALID_CREDENTAIL即登录失败，打印自己的user_id信息才是登录成功。
</div>

In [2]:
s = login()

b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'


### 二阶提升

In [2]:
fo_tracker = get_alphas_from_csv("output/simulated_alphas_ASI.csv.2025-05-13", 1.2, 0.7)
print(len(fo_tracker))

filtered_list = [
    item for item in fo_tracker
    if all(x not in item[1] for x in ['\n', 'trade']) and item[1].lower().count('group') < 2
]
print(len(filtered_list))
print(filtered_list[:3])

fo_layer = digest(filtered_list)
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory([expr], group_ops, "ASI"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

56
56
[['q511Zqj', 'ts_zscore(winsorize(ts_backfill(fnd23_caiv / fnd28_value_07240a, 120), std=4), 120)', 1.42, 0.1708, 0.91, 0.000822, '2025-05-12T20:04:25-04:00', 6], ['J3wEp7j', '-ts_zscore(winsorize(ts_backfill(fnd28_value_02101a / fnd6_chee, 120), std=4), 66)', -1.3, 0.2076, -1.06, -0.00134, '2025-05-12T20:21:07-04:00', 6], ['k87klPL', 'ts_zscore(winsorize(ts_backfill(fnd28_value_08005 / fnd28_value_09102, 120), std=4), 120)', 1.37, 0.0911, 0.81, 0.000951, '2025-05-12T21:35:59-04:00', 6]]
56
[['ts_zscore(winsorize(ts_backfill(fnd23_caiv / fnd28_value_07240a, 120), std=4), 120)', 6], ['-ts_zscore(winsorize(ts_backfill(fnd28_value_02101a / fnd6_chee, 120), std=4), 66)', 6], ['ts_zscore(winsorize(ts_backfill(fnd28_value_08005 / fnd28_value_09102, 120), std=4), 120)', 6]]
3696
[('group_rank(-ts_zscore(winsorize(ts_backfill(fnd6_newa2_ppent / fnd6_oiadpsa, 120), std=4), 120),densify(pv13_20_minvol_1m_sector))', 6), ("group_rank(-ts_rank(winsorize(ts_backfill(fnd23_2dls / fnd6_chee, 120

In [3]:
generate_pending_simulation_data(so_alpha_list,"SUBINDUSTRY", "ASI", "MINVOL1M", "append", 'alpha_list_pending_simulated_1.csv')

All simulation data written to output/alpha_list_pending_simulated_1.csv in append mode


### 三阶提升

In [4]:
## get promising alphas from second order to improve in the third order
#so_tracker = get_alphas("04-22", "04-23", 1.3, 0.8, "ASI", 5000, "track")
so_tracker = get_alphas_from_csv("output/simulated_alphas_ASI.csv.2025-05-13", 1.3, 0.8)

filtered_list = [
    item for item in so_tracker
    if all(x not in item[1] for x in ['\n', 'trade'])
]
print(len(filtered_list))
print(filtered_list[:3])
so_layer = digest(filtered_list)
th_alpha_list = []


for expr, decay in so_layer:
    for alpha in trade_when_factory("trade_when",expr,"ASI"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))

32
[['q511Zqj', 'ts_zscore(winsorize(ts_backfill(fnd23_caiv / fnd28_value_07240a, 120), std=4), 120)', 1.42, 0.1708, 0.91, 0.000822, '2025-05-12T20:04:25-04:00', 6], ['J3wEp7j', '-ts_zscore(winsorize(ts_backfill(fnd28_value_02101a / fnd6_chee, 120), std=4), 66)', -1.3, 0.2076, -1.06, -0.00134, '2025-05-12T20:21:07-04:00', 6], ['k87klPL', 'ts_zscore(winsorize(ts_backfill(fnd28_value_08005 / fnd28_value_09102, 120), std=4), 120)', 1.37, 0.0911, 0.81, 0.000951, '2025-05-12T21:35:59-04:00', 6]]
三阶表达式数量:1280


In [5]:
generate_pending_simulation_data(th_alpha_list,"SUBINDUSTRY", "ASI", "MINVOL1M", "append", 'alpha_list_pending_simulated_1.csv')

All simulation data written to output/alpha_list_pending_simulated_1.csv in append mode


## 3, 获取数据字段
<div style="margin-left: 20px;">
在官网Data页面中显示的为自己目前有权限的数据集，在数据集Description面板下可以看到dataset_id
</div>

In [3]:


import re

df = get_datafields(s, dataset_id = 'fundamental17', region='USA', universe='TOP3000', delay=1)
"""
target_field = df[df['description'].str.contains('EPS|Profit|Revenue|Income|Earning|Margin')]["id"].values.tolist()
print(target_field)
print(len(target_field))

anl15_df = get_datafields(s, dataset_id = 'analyst15', region='USA', universe='TOP3000', delay=1)
market_field = anl15_df[anl15_df['id'].str.contains('mean|pe|total|gro')]["id"].values.tolist()
market_field = [id_ for id_ in market_field if re.search('anl15_gr_|anl15_ind_|anl15_s_', id_)]
print(market_field[:10])
print(len(market_field))

sector_field = ['sta1_allc5', 'sta1_allc10', 'sta1_allc20', 'sta1_top3000c10', 'sta1_top500c10']
"""

'\ntarget_field = df[df[\'description\'].str.contains(\'EPS|Profit|Revenue|Income|Earning|Margin\')]["id"].values.tolist()\nprint(target_field)\nprint(len(target_field))\n\nanl15_df = get_datafields(s, dataset_id = \'analyst15\', region=\'USA\', universe=\'TOP3000\', delay=1)\nmarket_field = anl15_df[anl15_df[\'id\'].str.contains(\'mean|pe|total|gro\')]["id"].values.tolist()\nmarket_field = [id_ for id_ in market_field if re.search(\'anl15_gr_|anl15_ind_|anl15_s_\', id_)]\nprint(market_field[:10])\nprint(len(market_field))\n\nsector_field = [\'sta1_allc5\', \'sta1_allc10\', \'sta1_allc20\', \'sta1_top3000c10\', \'sta1_top500c10\']\n'

In [ ]:
target_fields_v = [
    "nws18_acb", "nws18_bam", "nws18_bee", "nws18_ber", "nws18_qcm", "nws18_qep", "nws18_ssc", "nws18_sse"
]
target_field_m = [
    "rp_css_earnings", "rp_css_revenue", "rp_ess_earnings", "rp_ess_revenue", "rp_nip_earnings"
]
target_fields = get_vec_fields(target_fields_v) + target_field_m
print(len(target_fields))

news36_df = get_datafields(s, dataset_id = 'news36', region='USA', universe='TOP3000', delay=1)
market_field_v = news36_df[news36_df['id'].str.contains('positive|negative|event')]["id"].values.tolist()
market_fields = get_vec_fields(market_field_v)
print(len(market_fields))
sector_fields = [
    "sta1_allc10", "sta1_allc20", "sta1_top3000c10", "sta1_top500c10"
]
news_days = [252]

In [2]:
exp_list = read_txt_to_list('data_set/ASID1fnd28.txt')
list = get_ids_from_csv_directory('data/ASI', "*_matrix.csv")

exp_list_filter = filter_expressions_by_list_b(exp_list, list)
pc_fields = process_list_datafields(exp_list_filter)
print(f"exp_list len {len(exp_list)}")
print(f"pc_fileds len {len(pc_fields)}")

Successfully read 5906 lines from data_set/ASID1fnd28.txt
从 data/ASI/model26_matrix.csv 提取 115 个 id: ['mdl26_5yr_hstrcl_grwth_rt', 'mdl26_actual_last_q_earnings', 'mdl26_actual_last_y_earnings', 'mdl26_average_revison', 'mdl26_blndd_grwth_ndstry_prcntl', 'mdl26_blndd_grwth_rt_smrtstmt', 'mdl26_blndd_grwth_sctr_prcntl', 'mdl26_chng_frm_52wk_hgh_prc', 'mdl26_chng_frm_52wk_lw_prc', 'mdl26_chng_rltv_t_cc_40_30', 'mdl26_chng_rltv_t_cntry_bnchmrk_30', 'mdl26_chng_rltv_t_dj_stxx_600_30', 'mdl26_chng_rltv_t_fts_100_30', 'mdl26_chng_rltv_t_hng_sng_30', 'mdl26_chng_rltv_t_js_ll_shr_30', 'mdl26_chng_rltv_t_msc_f_30', 'mdl26_chng_rltv_t_msc_mf_30', 'mdl26_chng_rltv_t_msc_pc_x_jp_30', 'mdl26_chng_rltv_t_msc_rp_30', 'mdl26_chng_rltv_t_ndstry_bnchmrk_30', 'mdl26_chng_rltv_t_r1000_grwth_30', 'mdl26_chng_rltv_t_rssll_2000_30', 'mdl26_chng_rltv_t_rssll_mdcp_30', 'mdl26_chng_rltv_t_sp_400_30', 'mdl26_chng_rltv_t_sp_500_30', 'mdl26_chng_rltv_t_sp_600_30', 'mdl26_chng_rltv_t_sx_100_30', 'mdl26_chng_rltv_t_

## 4，数据字段预处理

<div style="margin-left: 20px;">
1, matrix, vector 数据类型
</div>

<div style="margin-left: 20px;">
2, ts_backfill 回填缺失值，提高数据Coverage 
</div>

<div style="margin-left: 20px;">
2, winsorize 去极值
</div>

In [9]:
pc_fields = process_datafields(df)
len(pc_fields)

372

## 5, Alpha factory 
<div style="margin-left: 20px;">
在factory方法中将数据字段与操作符组装成alpha表达式
</div>

In [3]:
first_order = first_order_factory(pc_fields, ts_ops)
print(first_order[:10])
print(len(first_order))

['winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4)', 'ts_rank(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 5)', 'ts_rank(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 22)', 'ts_rank(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 66)', 'ts_rank(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 120)', 'ts_rank(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 240)', 'ts_zscore(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 5)', 'ts_zscore(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 22)', 'ts_zscore(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 66)', 'ts_zscore(winsorize(ts_backfill(rad / fnd28_value_01001a, 120), std=4), 120)']
323624


In [ ]:
first_order = generate_alpha_list(target_fields, market_fields, sector_fields, cap, news_days, ['residual_alpha','beta_alpha'])
print(first_order[:10])
print(len(first_order))

In [ ]:
first_order = generate_alpha_list(target_list, market_list, sector_list, cap, days, ['residual_alpha'])
print(first_order[:10])
print(len(first_order))

In [3]:
first_order = generate_alpha_list(
    base_target_list, 
    base_market_list, 
    cap_grouping_fields, 
    cap, 
    days, 
    ['base_alpha', 'base_rank_alpha'])
print(first_order[:10])
print(len(first_order))

["\n            target_data = winsorize(ts_backfill(close / dividend, 63), std=4.0);\n            alpha_1_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(cap), range='0.1, 1, 0.1'))), 252, rettype=2);\n            alpha_1_basic\n        ", "\n            target_data = winsorize(ts_backfill(close / dividend, 63), std=4.0);\n            alpha_1_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(cap), range='0.1, 1, 0.1'))), 252, rettype=2);\n            alpha = rank(alpha_1_basic);\n            alpha\n        ", "\n            target_data = winsorize(ts_backfill(close / dividend, 63), std=4.0);\n            alpha_2_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(cap), range='0.1, 1, 0.1'))), 504, rettype=2);\n            alpha_2_basic\n        ", "\n            target_data = winsorize(ts_backfill(close / dividend, 63), std=4.0);\n 

## 6, 回测前载入

<div style="margin-left: 20px;">
1, alpha表达式与初始decay配对
</div>

<div style="margin-left: 20px;">
2, random shuffle 
</div>

<div style="margin-left: 20px;">
2, Load task pool数据结构
</div>

In [4]:
# 赋予alpha表达式一个初始decay
init_decay = 6
fo_alpha_list = []

for alpha in first_order:
    fo_alpha_list.append((alpha, init_decay))

# 随机采样快速评估一个数据集的潜力
random.shuffle(fo_alpha_list)

print("数量: %s"%len(fo_alpha_list))
print(fo_alpha_list[:5])

数量: 323624
[('ts_sum(winsorize(ts_backfill(fnd6_accli / fnd6_prodv, 120), std=4), 5)', 6), ('ts_quantile(winsorize(ts_backfill(fnd17_lcxspebmtt / fnd17_ttmpelow, 120), std=4), 240)', 6), ('ts_std_dev(winsorize(ts_backfill(fnd6_newq_loq / fnd6_prvy, 120), std=4), 120)', 6), ('ts_scale(winsorize(ts_backfill(fnd17_inmtt / fnd6_newq_lltq, 120), std=4), 22)', 6), ('ts_delay(winsorize(ts_backfill(fnd17_lcxspebmtt / fnd17_ttmpelow, 120), std=4), 66)', 6)]


In [6]:
# 赋予alpha表达式一个初始decay
init_decay = 6
fo_alpha_list_usa = []

for alpha in first_order:
    fo_alpha_list_usa.append((alpha, init_decay))

# 随机采样快速评估一个数据集的潜力
random.shuffle(fo_alpha_list_usa)

print("数量: %s"%len(fo_alpha_list_usa))
print(fo_alpha_list_usa[:5])

数量: 43680
[('ts_scale(winsorize(ts_backfill(fnd17_tbea / fnd23_swds, 120), std=4), 5)', 6), ('ts_quantile(winsorize(ts_backfill(fnd17_qtaxpd / fnd23_ntiv, 120), std=4), 66)', 6), ('ts_arg_min(winsorize(ts_backfill(fnd17_tbemtt / fnd23_caiv, 120), std=4), 120)', 6), ('ts_mean(winsorize(ts_backfill(fnd17_qroipct / fnd23_nins, 120), std=4), 66)', 6), ('ts_mean(winsorize(ts_backfill(fnd17_rhsfcq / fnd28_value_05101, 120), std=4), 120)', 6)]


In [7]:
generate_pending_simulation_data(fo_alpha_list[:50000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_2.csv")
generate_pending_simulation_data(fo_alpha_list[50000 : 100000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_3.csv")
generate_pending_simulation_data(fo_alpha_list[100000 : 150000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_4.csv")
generate_pending_simulation_data(fo_alpha_list[150000 : 200000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_5.csv")
generate_pending_simulation_data(fo_alpha_list[200000 : 250000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_6.csv")
generate_pending_simulation_data(fo_alpha_list[250000 : 300000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_10.csv")
generate_pending_simulation_data(fo_alpha_list[300000 :],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_11.csv")
"""
generate_pending_simulation_data(fo_alpha_list[:6],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite")
generate_pending_simulation_data(fo_alpha_list_usa[:6],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[7:12],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[7:12],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[13:18],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[13:18],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[19:24],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[19:24],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[25:30],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[25:30],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[31:36],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[31:36],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[37:42],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[37:42],"SUBINDUSTRY", "USA", "TOP3000", "append")
"""


Deleted existing file: output/alpha_list_pending_simulated_2.csv
All simulation data written to output/alpha_list_pending_simulated_2.csv in overwrite mode
Deleted existing file: output/alpha_list_pending_simulated_3.csv
All simulation data written to output/alpha_list_pending_simulated_3.csv in overwrite mode
Deleted existing file: output/alpha_list_pending_simulated_4.csv
All simulation data written to output/alpha_list_pending_simulated_4.csv in overwrite mode
Deleted existing file: output/alpha_list_pending_simulated_5.csv
All simulation data written to output/alpha_list_pending_simulated_5.csv in overwrite mode
Deleted existing file: output/alpha_list_pending_simulated_6.csv
All simulation data written to output/alpha_list_pending_simulated_6.csv in overwrite mode
All simulation data written to output/alpha_list_pending_simulated_10.csv in overwrite mode
All simulation data written to output/alpha_list_pending_simulated_11.csv in overwrite mode


'\ngenerate_pending_simulation_data(fo_alpha_list[:6],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite")\ngenerate_pending_simulation_data(fo_alpha_list_usa[:6],"SUBINDUSTRY", "USA", "TOP3000", "append")\ngenerate_pending_simulation_data(fo_alpha_list[7:12],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")\ngenerate_pending_simulation_data(fo_alpha_list_usa[7:12],"SUBINDUSTRY", "USA", "TOP3000", "append")\ngenerate_pending_simulation_data(fo_alpha_list[13:18],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")\ngenerate_pending_simulation_data(fo_alpha_list_usa[13:18],"SUBINDUSTRY", "USA", "TOP3000", "append")\ngenerate_pending_simulation_data(fo_alpha_list[19:24],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")\ngenerate_pending_simulation_data(fo_alpha_list_usa[19:24],"SUBINDUSTRY", "USA", "TOP3000", "append")\ngenerate_pending_simulation_data(fo_alpha_list[25:30],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")\ngenerate_pending_simulation_data(fo_alpha_list_usa[25:30],"SUBINDUSTRY", "USA", "TOP3000", "append"

In [ ]:
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOP1000", "append")
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOP500", "append")
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOP200", "append")
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOPSP500", "append")

In [2]:
# 文件路径
file_a_path = os.path.join('/Users/sujianan/py/alpha_simulator_bot/data', 'alpha_list_pending_simulated.csv')
file_b_path = os.path.join('/Users/sujianan/py/consultant/consultant/output', 'alpha_list_pending_simulated.csv')

# 合并文件
merge_csv_files(file_a_path, file_b_path, 'append')

文件合并完成（模式：append），结果已保存为：/Users/sujianan/py/alpha_simulator_bot/data/alpha_list_pending_simulated.csv


In [8]:
# Load alphas to task pools
fo_pools = load_task_pool(fo_alpha_list, 10, 10)
print(fo_pools[0])

[[('ts_delta(winsorize(ts_backfill(fnd13_adep, 120), std=4), 120)', 6), ('ts_delay(winsorize(ts_backfill(fnd13_rkdbalancesheetq_crta, 120), std=4), 66)', 6), ('ts_arg_max(winsorize(ts_backfill(fnd13_pxtl, 120), std=4), 120)', 6), ('ts_scale(winsorize(ts_backfill(fnd13_mpns, 120), std=4), 66)', 6), ('ts_quantile(winsorize(ts_backfill(fnd13_viat, 120), std=4), 120)', 6), ('ts_mean(winsorize(ts_backfill(fnd13_dlcl, 120), std=4), 240)', 6), ('ts_scale(winsorize(ts_backfill(fnd13_acoa, 120), std=4), 240)', 6), ('ts_arg_min(winsorize(ts_backfill(fnd13_ists, 120), std=4), 22)', 6), ('ts_sum(winsorize(ts_backfill(fnd13_agse, 120), std=4), 66)', 6), ('ts_mean(winsorize(ts_backfill(fnd13_rkdincomestatementq_stnr, 120), std=4), 22)', 6)], [('ts_quantile(winsorize(ts_backfill(fnd13_soca, 120), std=4), 22)', 6), ('ts_std_dev(winsorize(ts_backfill(fnd13_lcol, 120), std=4), 66)', 6), ('ts_quantile(winsorize(ts_backfill(fnd13_aing, 120), std=4), 22)', 6), ('ts_zscore(winsorize(ts_backfill(fnd13_rkdbal

## 7, 回测

In [9]:
# Simulate First Order
multi_simulate(fo_pools, "SUBINDUSTRY", "USA", "TOP3000", 0)

b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
sim_data_list:[{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 6, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'testPeriod': 'P0Y', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'ts_delta(winsorize(ts_backfill(fnd13_adep, 120), std=4), 120)'}, {'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 6, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'testPeriod': 'P0Y', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'ts_delay(winsorize(ts_backfill(fnd13_rkdbalancesheetq_crta, 120), std

KeyboardInterrupt: 

## 8, 筛选Alpha


<div style="margin-left: 20px;">
1, get_alpha：截取有潜力提升表现至可以提交的alpha进入下一阶
</div>

<div style="margin-left: 20px;">
2, 剪枝Prune：精减相似alpha，提高回测资源利用率
</div>

In [3]:
## get promising alphas to improve in the next order
fo_tracker = get_alphas("04-22", "04-23", 1.2, 0.7, "ASI", 5000, "track")
print(len(fo_tracker))
filtered_list = [
    item for item in fo_tracker
    if all(x not in item[1] for x in ['group', '\n', 'trade'])
]
print(len(filtered_list))
print(filtered_list[:3])

b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
0
['e5Ggp7g', 'group_neutralize(-ts_zscore(winsorize(ts_backfill(fnd17_atotltd / fnd6_chee, 120), std=4), 240),densify(sta1_minvol1mc50))', 2.45, 0.1094, 1.92, 0.001396, '2025-04-22T17:31:31-04:00', 6]
['9xMrvqe', 'group_neutralize(-ts_zscore(winsorize(ts_backfill(fnd17_atotltd / fnd6_chee, 120), std=4), 240),densify(sta1_minvol1mc20))', 2.41, 0.1063, 1.89, 0.001442, '2025-04-22T19:09:17-04:00', 6]
['WOzRY6o', 'group_neutralize(-ts_zscore(winsorize(ts_backfill(fnd17_atotltd / fnd6_chee, 120), std=4), 240),densify(pv13_2_f4_g3_minvol_1m_sector))', 2.35, 0.1098, 1.82, 0.001368, '2025-04-22T22:26:17-04:00', 6]
['NNz7nqq', 'group_neutralize(-ts_zscore(winsorize(ts_backfill(fnd17_atotltd / fnd6_chee, 120), std=4), 240),densify(pv13_5_f3_g2_minvol_1m_sector))', 2.33, 0.1056, 1.81, 0.001423, '2025-04-22T19:

In [4]:
fo_tracker = get_alphas_with_universe_region("04-09", "04-11", 1.2, 0.7, 'GLB', 300, "track")
filtered_list = [item for item in fo_tracker if '\n' not in item[1]]
print(len(filtered_list))
for item in filtered_list:
    print(item)

b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
0
100
200
count: 0
0


#### Prune 剪枝

In [9]:
#fo_layer = prune(filtered_list, 'fnd17', 5)
fo_layer = digest(filtered_list)
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

#fo_exp_layer = transform_expressions(fo_layer)
#print(len(fo_exp_layer))
#print(fo_exp_layer[:3])


26
[['ts_zscore(winsorize(ts_backfill(fnd17_fcf1a / fnd23_swds, 120), std=4), 240)', 6], ['ts_zscore(winsorize(ts_backfill(fnd17_fcf1a / fnd23_dltv, 120), std=4), 120)', 6], ['ts_zscore(winsorize(ts_backfill(fnd17_hsca / fnd23_dlts, 120), std=4), 240)', 6]]


In [10]:
fo_layer = prune_with_universe(filtered_list, 'ts_backfill', 5, 'TOP3000')
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

62
[["\n            target_data = winsorize(ts_backfill(implied_volatility_put_90, 63), std=4.0);\n            alpha_2088_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(cap), range='0.1, 1, 0.1'))), 756, rettype=2);\n            alpha = rank(alpha_2088_basic);\n            alpha\n        ", 6], ["\n            target_data = winsorize(ts_backfill(implied_volatility_put_150, 63), std=4.0);\n            alpha_2241_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(close * volume), range='0.1, 1, 0.1'))), 756, rettype=2);\n            alpha = rank(alpha_2241_basic);\n            alpha\n        ", 6], ["\n            target_data = winsorize(ts_backfill(implied_volatility_put_60, 63), std=4.0);\n            alpha_2019_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(cap), range='0.1, 1, 0.1'))), 756, rettype=2);\n            alpha = ra

In [4]:

# 主程序
deduped_alphas = deduplicate_alphas(filtered_list)
print(f"Original Alphas: {len(alphas)}, After Deduplication: {len(deduped_alphas)}")

param_combinations = generate_param_combinations()
print(len(param_combinations))
print(param_combinations[:3])
all_transformed_alphas = []

for alpha in deduped_alphas:
    for params in param_combinations:
        transformed_alpha = generate_transformed_alpha(alpha, **params)
        all_transformed_alphas.append(transformed_alpha)

unique_transformed_alphas = deduplicate_transformed_alphas(all_transformed_alphas)

# 输出前5个结果
for i, ta in enumerate(unique_transformed_alphas[:5]):
    print(f"Combination {i + 1}:")
    print(f"Alpha ID: {ta['alpha_id']}")
    print(f"Parameters: {ta['params']}")
    print(f"Transformed Expression:\n{ta['transformed_exp']}")
    print(f"Original Sharpe: {ta['original_sharpe']}, Original Fitness: {ta['original_fitness']}")
    print("-" * 70)

# 保存所有唯一结果
import json
with open('unique_transformed_alphas.json', 'w') as f:
    json.dump(unique_transformed_alphas, f, indent=4)
print(f"Total unique combinations generated: {len(unique_transformed_alphas)}")

Original Alphas: 3, After Deduplication: 46
612
[{'data_types': ['implied_volatility_put'], 'vol_terms': ['90', '180'], 'return_type': 'ts_mean(returns, 21)', 'type_weights': [1.0], 'term_weights': [0.5, 0.5], 'return_weight': 0.3, 'backfill_days': 63, 'winsorize_std': 4.0}, {'data_types': ['implied_volatility_put'], 'vol_terms': ['90', '180'], 'return_type': 'ts_mean(returns, 21)', 'type_weights': [1.0], 'term_weights': [0.6, 0.4], 'return_weight': 0.3, 'backfill_days': 63, 'winsorize_std': 4.0}, {'data_types': ['implied_volatility_put'], 'vol_terms': ['90', '180'], 'return_type': 'ts_mean(returns, 21)', 'type_weights': [1.0], 'term_weights': [0.4, 0.6], 'return_weight': 0.3, 'backfill_days': 63, 'winsorize_std': 4.0}]
Combination 1:
Alpha ID: L57a0d2
Parameters: {'data_types': ['implied_volatility_put'], 'vol_terms': ['90', '180'], 'return_type': 'ts_mean(returns, 21)', 'type_weights': [1.0], 'term_weights': [0.5, 0.5], 'return_weight': 0.3, 'backfill_days': 63, 'winsorize_std': 4.0}

## 9, 二阶提升
### ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [10]:
so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory([expr], group_ops, "ASI"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

1716
[('group_neutralize(ts_zscore(winsorize(ts_backfill(fnd23_fbbs / fnd28_value_05191q, 120), std=4), 120),densify(sta1_minvol1m_normc50))', 6), ('group_zscore(ts_rank(winsorize(ts_backfill(fnd17_fcfmtt / fnd23_sonm, 120), std=4), 22),densify(pv13_20_minvol_1m_sector))', 6), ('group_rank(-ts_zscore(winsorize(ts_backfill(fnd23_eaca / fnd6_chee, 120), std=4), 240),densify(pv13_5_f3_g2_minvol_1m_sector))', 6)]


In [9]:
so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory_for_multi_line([expr], group_ops, "USA"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

138
[('target_data = winsorize(ts_backfill(returns, 63), std=4.0);\n            market_data = winsorize(ts_backfill(pv53_logreturns5_normalized, 63), std=4.0);\n            beta = ts_regression(target_data, group_mean(market_data, log(ts_mean(cap, 21)), densify(sector)), 66, rettype=2);\n            predicted_return = beta * market_data;\n            alpha_1_residual = - target_data + predicted_return;\n            alpha = group_zscore(alpha_1_residual,densify(sta2_top3000_fact3_c50));\n            alpha', 18), ('target_data = winsorize(ts_backfill(returns, 63), std=4.0);\n            market_data = winsorize(ts_backfill(pv53_logreturns5_normalized, 63), std=4.0);\n            beta = ts_regression(target_data, group_mean(market_data, log(ts_mean(cap, 21)), densify(sector)), 66, rettype=2);\n            predicted_return = beta * market_data;\n            alpha_1_residual = - target_data + predicted_return;\n            alpha = group_rank(alpha_1_residual,densify(sta1_allc5));\n          

In [5]:
so_alpha_list = []
so_decay = 6
alpha_list = [item['transformed_exp'] for item in unique_transformed_alphas]
for alpha in alpha_list:
    so_alpha_list.append((alpha,so_decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

28152
[("target_data = winsorize(0.08000000000000002 * ts_backfill(implied_volatility_put_90, 63) + 0.12 * ts_backfill(implied_volatility_put_180, 63) + 0.2 * ts_backfill(call_breakeven_90, 63) + 0.3 * ts_backfill(call_breakeven_180, 63) + 0.12 * ts_backfill(forward_price_90, 63) + 0.18 * ts_backfill(forward_price_180, 63) + (0.3 * ts_backfill(ts_mean(returns, 21), 63)), std=4.0);\n        alpha_2517_basic = ts_regression(target_data, group_mean(target_data, log(ts_mean(cap, 21)), densify(bucket(rank(close * volume), range='0.1, 1, 0.1'))), 756, rettype=2); alpha = rank(alpha_2517_basic); alpha", 6), ("target_data = winsorize(0.1 * ts_backfill(implied_volatility_put_20, 63) + 0.2 * ts_backfill(implied_volatility_put_60, 63) + 0.2 * ts_backfill(implied_volatility_put_90, 63) + 0.1 * ts_backfill(call_breakeven_20, 63) + 0.2 * ts_backfill(call_breakeven_60, 63) + 0.2 * ts_backfill(call_breakeven_90, 63) + (0.3 * ts_backfill(ts_mean(returns, 21), 63)), std=4.0);\n        alpha_5688_basic =

In [14]:
df = get_datafields(s, dataset_id = 'pv29', region='USA', universe='TOP3000', delay=1)

group_field = df[df['type'] == 'GROUP']["id"].values.tolist()
print(group_field)
print(len(group_field))

so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
    lines = expr.strip().split('\n')
    # 获取最后一行并去除首尾空格
    last_line = lines[-1].strip()

    # 排除最后一行是 alpha 或 trade_alpha 的情况
    if last_line in ['alpha', 'trade_alpha']:
        continue
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory_for_multi_line_by_groups([expr], group_ops, group_field, "USA"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

['sta1_allc10', 'sta1_allc2', 'sta1_allc20', 'sta1_allc5', 'sta1_top1000c10', 'sta1_top1000c2', 'sta1_top1000c20', 'sta1_top1000c5', 'sta1_top1000c50', 'sta1_top200c10', 'sta1_top200c2', 'sta1_top200c20', 'sta1_top200c5', 'sta1_top200c50', 'sta1_top3000c10', 'sta1_top3000c2', 'sta1_top3000c20', 'sta1_top3000c5', 'sta1_top3000c50', 'sta1_top500c10', 'sta1_top500c2', 'sta1_top500c20', 'sta1_top500c5', 'sta1_top500c50']
24
504
[('target_data = winsorize(ts_backfill(returns, 63), std=4.0);\n            market_data = winsorize(ts_backfill(pv53_logreturns5_normalized, 63), std=4.0);\n            beta = ts_regression(target_data, group_mean(market_data, log(ts_mean(cap, 21)), densify(industry)), 756, rettype=2);\n            predicted_return = beta * market_data;\n            alpha_10_residual = - target_data + predicted_return;\n            alpha = group_rank(alpha_10_residual,densify(sta1_top500c5));\n            alpha', 21), ('target_data = winsorize(ts_backfill(returns, 63), std=4.0);\n  

In [11]:
generate_pending_simulation_data(so_alpha_list,"SUBINDUSTRY", "ASI", "MINVOL1M", "append", 'alpha_list_pending_simulated_1.csv')

All simulation data written to output/alpha_list_pending_simulated_1.csv in append mode


In [3]:
# 文件路径
file_a_path = os.path.join('/Users/sujianan/py/alpha_simulator_bot/data', 'alpha_list_pending_simulated.csv')
file_b_path = os.path.join('/Users/sujianan/py/consultant/consultant/output', 'alpha_list_pending_simulated.csv')

# 合并文件
merge_csv_files(file_a_path, file_b_path, 'append')

文件合并完成（模式：append），结果已保存为：/Users/sujianan/py/alpha_simulator_bot/data/alpha_list_pending_simulated.csv


### Simulate second order

In [ ]:
so_pools = load_task_pool(so_alpha_list, 10, 10)
multi_simulate(so_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## 10，三阶提升
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [12]:
## get promising alphas from second order to improve in the third order
#so_tracker = get_alphas("04-22", "04-23", 1.3, 0.8, "ASI", 5000, "track")
so_tracker = get_alphas_from_csv("output/simulated_alphas_ASI.csv.2025-05-02", 1.3, 0.8)

filtered_list = [
    item for item in so_tracker
    if all(x not in item[1] for x in ['\n', 'trade'])
]
print(len(filtered_list))
print(filtered_list[:3])
so_layer = digest(filtered_list)
th_alpha_list = []


for expr, decay in so_layer:
    for alpha in trade_when_factory("trade_when",expr,"ASI"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))

18
[['maqmXO6', 'ts_zscore(winsorize(ts_backfill(fnd17_fcf1a / fnd23_dltv, 120), std=4), 120)', 1.4, 0.1448, 0.84, 0.000728, '2025-05-01T21:11:34-04:00', 6], ['jan8WoZ', 'ts_zscore(winsorize(ts_backfill(fnd17_hsca / fnd23_dlts, 120), std=4), 240)', 1.34, 0.1177, 0.82, 0.000796, '2025-05-01T21:21:44-04:00', 6], ['Y1e9k8v', 'ts_zscore(winsorize(ts_backfill(fnd17_inmtt / fnd28_value_05301q, 120), std=4), 66)', 1.48, 0.1599, 0.85, 0.000662, '2025-04-25T14:04:27-04:00', 6]]
三阶表达式数量:720


In [3]:
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas_with_universe_region("04-01", "04-04", 1.3, 0.8, "USA", 200, "submit")

so_layer = prune_with_universe(so_tracker, 'fnd6', 5, 'TOP500')
th_alpha_list = []

for expr, decay in so_layer:
    for alpha in trade_when_factory_for_multi_line("trade_when",expr,"USA"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))
print(th_alpha_list[:3])

b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
0
['2jaepVJ', 'target_data = winsorize(ts_backfill((close - ts_delay(close, 1)) / ts_delay(close, 1), 63), std=4.0);\n            market_data = winsorize(ts_backfill(pv53_logreturns5_normalized, 63), std=4.0);\n            beta = ts_regression(target_data, group_mean(market_data, log(ts_mean(cap, 21)), densify(industry)), 756, rettype=2);\n            predicted_return = beta * market_data;\n            alpha_85_residual = - target_data + predicted_return;\n            alpha = group_zscore(alpha_85_residual,densify(pv13_h_min2_3000_sector));\n            alpha', 1.79, 0.3978, 1.03, 0.000664, 'TOP3000', 'USA', '2025-04-03T02:17:09-04:00', 21, 25]
['x1PWxal', 'target_data = winsorize(ts_backfill(returns, 63), std=4.0);\n            market_data = winsorize(ts_backfill(pv53_logreturns5_normalized, 63), st

In [13]:
generate_pending_simulation_data(th_alpha_list,"SUBINDUSTRY", "ASI", "MINVOL1M", "append", 'alpha_list_pending_simulated_1.csv')

All simulation data written to output/alpha_list_pending_simulated_1.csv in append mode


### Simulate Third Order

In [ ]:
# Simulate third order
th_pools = load_task_pool(th_alpha_list, 10, 9)
multi_simulate(th_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## 11, 获取可提交的Alpha

<div style="margin-left: 20px;">
1, 拉取sharpe,fitness达到提交要求的alpha
</div>

<div style="margin-left: 20px;">
2, Check Submission：检查其他Test是否达到要求
</div>

<div style="margin-left: 20px;">
2, view_alphas 对可以提交的alpha进行排序
</div>


In [ ]:
# 1.58 sharpe, 1 fitness, "submit"参数
th_tracker = get_alphas("07-16", "07-20", 1.58, 1, "USA", 200, "submit")

In [ ]:
## 将get的alpha的id取出至stone_bag，用api check submission
stone_bag = []
for alpha in th_tracker:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []
check_submission(stone_bag, gold_bag, 0)

In [ ]:
# 打印可提交的alpha信息并按sharpe排序，在网页上找到alpha手动提交
view_alphas(gold_bag)

## 12, 微调可以提交的alpha

<div style="margin-left: 20px;">
1, 得到更好的表现
</div>
<div style="margin-left: 40px;">
调整中性化，操作符参数，Decay
</div>

<div style="margin-left: 20px;">
2, Alpha质量评估
</div>

<div style="margin-left: 40px;">
performance comparison，turnover，margin
</div>

<div style="margin-left: 20px;">
3, 鲁棒性评估，防止过拟合
</div>

<div style="margin-left: 40px;">
更改中性化，Rank，Binary Test...
</div>

### Appendix

In [ ]:
# 模板构建Factory实例

def template_factory(sent_fields, option_fields):
    alpha_list = []
    for sent_field in sent_fields:
        for opt_field in option_fields:
            alpha_list.append("log(1+sigmoid(ts_zscore(%s,30))*sigmoid(ts_zscore(%s,30))"%(sent_field, opt_field))
    return alpha_list 

opt_df = get_datafields(s, dataset_id = 'option8', region='USA', universe='TOP3000', delay=1)
opt_fields = opt_df[opt_df['type'] == "MATRIX"]["id"].tolist()
print(opt_fields)

sent_df = get_datafields(s, dataset_id = 'sentiment1', region='USA', universe='TOP3000', delay=1)
sent_fields = sent_df[sent_df['type'] == "MATRIX"]["id"].tolist()
print(sent_fields)

alpha_list = template_factory(sent_fields, opt_fields)
print(alpha_list)